In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
ratings_df = pd.read_csv('ratingsProcessed.csv')
train_rating_df = pd.read_csv('train_rating.csv')
test_rating_df = pd.read_csv('test_rating.csv')
userNo,movieNo = ratings_df['userId'].max()+1,ratings_df['movieId'].max()+1
testNo = np.shape(test_rating_df)[0]

In [ ]:
userNo,movieNo,testNo

In [ ]:
train_global_rating_mean = np.mean(train_rating_df["rating"])

In [ ]:
train_rating = np.zeros((userNo,movieNo))
for index ,row in train_rating_df.iterrows():
    train_rating[int(row['userId']),int(row['movieId'])] = row['rating']
test_rating = np.zeros((userNo,movieNo))
for index ,row in test_rating_df.iterrows():
    test_rating[int(row['userId']),int(row['movieId'])] = row['rating']

In [ ]:
train_record = train_rating>0
train_record = np.array(train_record,dtype=int)
test_record = test_rating>0
test_record = np.array(test_record,dtype=int)

In [ ]:
f = 100
lamda = 0.01

In [ ]:
def createBSVD(userNo,bookNo,f):
    BIAS_u = train_global_rating_mean
    BIAS_bu = tf.Variable(tf.zeros((userNo,1),dtype=tf.float32))
    BIAS_bi = tf.Variable(tf.zeros((1,bookNo),dtype=tf.float32))
    BIAS_P = tf.Variable(tf.random_normal([userNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    BIAS_Q = tf.Variable(tf.random_normal([movieNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    BIAS_predict_rating = BIAS_u + BIAS_bu + BIAS_bi + tf.matmul(BIAS_P,BIAS_Q,transpose_b=True)
    BIAS_loss = tf.reduce_sum(((train_rating-BIAS_predict_rating)*train_record)**2) + lamda*((tf.reduce_sum(BIAS_P)**2) + (tf.reduce_sum(BIAS_Q)**2) + (tf.reduce_sum(BIAS_bu)**2) + (tf.reduce_sum(BIAS_bi)**2))
    return BIAS_loss,BIAS_predict_rating